### Initialize the hotel codes and environment here

In [ ]:
import requests
import json
import os
import pymongo
from tabulate import tabulate
from datetime import datetime, timedelta
import pandas as pd
from collections import OrderedDict
from dotenv import load_dotenv, dotenv_values

load_dotenv("app.env")

REWARRDS_DB = os.getenv("REWARDS_CONNECTION_URI")
STAGING_DB = os.getenv("STAGING_CONNECTION_URI")
DEV_DB = os.getenv("DEV_CONNECTION_URI")
HOTELS_PROD_DB = os.getenv("PROD_CONNECTION_URI")
VEHO_PROD_DB = os.getenv("VEHO_PROD_CONNECTION_URI")
TRIPLAZE_JWT = os.getenv("TRIPLAZE_JWT")


def check_env(key):
    env = os.getenv(key)
    print(env)


def validate_environment(environment):
    if environment not in ["rewards", "staging", "dev", "production"]:
        return False
    return True


def add_thru_api(triplaze_hotel_codes, environment):
    print("Initiating API Call")
    if environment == "production":
        url = "https://hotels.pickyourtrail.com/content/TRIPLAZE"
    else:
        url = f"http://{environment}.wwmibprivate.com:9090/content/TRIPLAZE"

    headers = {"Content-Type": "application/json"}

    payload = json.dumps(triplaze_hotel_codes)

    print(f"Executing API Call, for url: {url}, headers: {headers}, payload: {payload}")
    response = requests.put(url, headers=headers, data=payload)
    print(f"API Call response: {response.text}, status code: {response.status_code}")

    if response.status_code != 200:
        raise Exception(f"API Call failed with status code: {response.status_code}")


def add_hotels_to_db(triplaze_hotel_codes, environment):
    environment_ok = validate_environment(environment)
    if not environment_ok:
        raise Exception(f"Invalid environment: {environment}")

    if not triplaze_hotel_codes:
        raise Exception("Triplaze hotel codes cannot be empty")

    print(f"Adding triplaze hotels to {environment} database")
    print(f"Triplaze hotel codes: {triplaze_hotel_codes}")

    add_thru_api(triplaze_hotel_codes, environment)
    print(f"Successfully added triplaze hotels to {environment} database")

    # cities = get_distinct_cities(triplaze_hotel_codes, environment)
    # generate_fuzzy_scripts(cities)


def get_uri_for_environment(environment):
    environment_ok = validate_environment(environment)
    if not environment_ok:
        return None
    uri_by_environment = {
        "rewards": REWARRDS_DB,
        "staging": STAGING_DB,
        "dev": DEV_DB,
        "production": HOTELS_PROD_DB,
    }
    return uri_by_environment[environment]


def get_uri_for_veho_environment(environment):
    environment_ok = validate_environment(environment)
    if not environment_ok:
        return None
    if environment == "production":
        return VEHO_PROD_DB
    return get_uri_for_environment(environment)


def get_distinct_cities(triplaze_hotel_codes, environment):
    print(
        f"Getting cities for triplaze hotel codes: {triplaze_hotel_codes} in {environment} environment"
    )
    print(f"Establishng connection to {environment} database")
    uri = get_uri_for_environment(environment)

    if uri:
        client = pymongo.MongoClient(uri)
        db = client["hotels"]
        collection = db["triplaze_hotel"]

        cities = collection.find({"hotelCode": {"$in": triplaze_hotel_codes}}).distinct(
            "cityName"
        )
        if cities:
            print(f"Cities found: {cities}")
            return cities
        else:
            print("Cities not found")
            return []
    else:
        raise Exception(f"Invalid environment: {environment}")


def generate_fuzzy_sheet(city):
    url = f"https://hotels.pickyourtrail.com/content/fuzzy?cityName={city}&provider=TRIPLAZE"

    response = requests.get(url)
    if response.status_code == 200:
        if response.headers.get("Content-Type") == "application/vnd.ms-excel":
            # Get the file name from the URL
            file_name = f"{city.lower()}_fuzzy_results.xlsx"
            # Specify the local folder path to save the Excel file
            save_folder = "./resources/fuzzy-sheets/"
            # Create the save folder if it doesn't exist
            os.makedirs(save_folder, exist_ok=True)
            # Save the Excel file to the local folder
            save_path = os.path.join(save_folder, file_name)
            with open(save_path, "wb") as file:
                file.write(response.content)
            print(f"Excel file saved to: {save_path}")
            print(f"Fuzzy sheet generated successfully for city {city}")
            return True
    return False


def generate_db_script(city):
    print(f"Generating DB script for city {city}")
    file_name = f"{city.lower()}_fuzzy_results.xlsx"

    url = "https://hotels.pickyourtrail.com/content/fuzzy/data"
    payload = {"provider": "TRIPLAZE"}
    headers = {}
    files = [
        (
            "file",
            (
                file_name,
                open(f"./resources/fuzzy-sheets/{file_name}", "rb"),
                "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
            ),
        )
    ]

    response = requests.post(url, headers=headers, data=payload, files=files)
    if response.status_code == 200:
        print(f"DB script generated successfully for city {city}")
        if response.headers.get("Content-Type") == "text/plain":
            # Get the file name from the URL
            file_name = f"{city.lower()}_fuzzy_script.mongodb.js"
            # Specify the local folder path to save the Excel file
            save_folder = "./resources/fuzzy-scripts/"
            # Create the save folder if it doesn't exist
            os.makedirs(save_folder, exist_ok=True)
            # Save the Excel file to the local folder
            save_path = os.path.join(save_folder, file_name)
            with open(save_path, "wb") as file:
                file.write(response.content)
            print(f"Excel file saved to: {save_path}")
            return True
    return False


def generate_fuzzy_scripts(cities):
    print(f"Generating fuzzy scripts for cities {cities}")

    for city in cities:
        status = generate_fuzzy_sheet(city)
        if status:
            script_status = generate_db_script(city)
            if script_status:
                print(f"Fuzzy script generated successfully for city {city}")


def get_hotel_active_status_from_portal(hotel_code):
    url = f"https://pyt-extapi-live.tiniva.com/api/ext-property/dashboardProperty?supplierId=1134&status=ALL&pageNo=1&rows=50&productId={hotel_code}"

    headers = {
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
        "Origin": "https://pyt-ext-live.tiniva.com",
        "Pragma": "no-cache",
        "Referer": "https://pyt-ext-live.tiniva.com/",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-site",
        "Sec-GPC": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        "jwt": TRIPLAZE_JWT,
        "sec-ch-ua": '"Brave";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"macOS"',
    }
    repsonse = requests.request("GET", url, headers=headers, data={})
    data = json.loads(repsonse.text)
    df = pd.DataFrame(data["hotelList"])
    return df


def get_portal_status(hotel_codes):
    print("Triplaze Hotels Portal Status")
    df_list = []
    for hotel_code in hotel_codes:
        df_list.append(get_hotel_active_status_from_portal(hotel_code))

    df = pd.concat(df_list)
    inactive_hotels = df.loc[df["status"] == "InActive", "productId"].to_list()
    print(tabulate(df, headers="keys", tablefmt="pretty"))
    print("Inactive Hotels")
    print(inactive_hotels)


def activate_triplaze_hotel(hotel_code):
    url = f"https://pyt-extapi-live.tiniva.com/api/ext-property/changeProductStatus?enable=Active&productId={hotel_code}"
    headers = {
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
        "Origin": "https://pyt-ext-live.tiniva.com",
        "Pragma": "no-cache",
        "Referer": "https://pyt-ext-live.tiniva.com/",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-site",
        "Sec-GPC": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
        "jwt": TRIPLAZE_JWT,
        "sec-ch-ua": '"Not/A)Brand";v="8", "Chromium";v="126", "Brave";v="126"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"macOS"',
    }

    response = requests.request("GET", url, headers=headers, data={})
    if response.status_code == 200:
        data = json.loads(response.text)
        return data
    return None


def refresh_hotels_for_city(payload, environment):
    if environment == "production":
        url = "https://hotels.pickyourtrail.com/hotel"
    else:
        url = f"http://{environment}.wwmibprivate.com:9090/hotel"

    payload = json.dumps(payload)
    headers = {"Content-Type": "application/json"}

    print(url, headers, payload)
    response = requests.post(url, headers=headers, data=payload)
    if response.status_code == 200:
        data = json.loads(response.text)
        print(data)
    else:
        print("Error performing city refresh")


def get_rooms_map(hotel_codes, source_provider="EXPEDIA"):
    collection_ref = {"EXPEDIA": "rapid_hotel", "AGODA": "agoda_hotel"}
    hotel = (
        client.get_database("hotels")
        .get_collection("hotel")
        .find_one(
            {"hotelCodeBySourceProvider.TRIPLAZE": hotel_codes[0]},
            {
                "hotelCodeBySourceProvider": 1,
            },
        )
    )
    global expedia_hotel_code
    expedia_hotel_code = hotel["hotelCodeBySourceProvider"][source_provider]
    expedia_hotel = (
        client.get_database("hotels")
        .get_collection(collection_ref[source_provider])
        .find_one({"hotelCode": expedia_hotel_code}, {"roomsMap": 1, "_id": 0})
    )

    triplaze_hotel = (
        client.get_database("hotels")
        .get_collection("triplaze_hotel")
        .find_one({"hotelCode": hotel_codes[0]}, {"roomsMap": 1, "_id": 0})
    )

    triplaze_rooms = {}
    for code, room_obj in triplaze_hotel["roomsMap"].items():
        triplaze_rooms[room_obj["roomId"]] = room_obj["name"]
    print("TRIPLAZE HOTEL")
    triplaze_rooms_df = pd.DataFrame([triplaze_rooms])
    triplaze_rooms_transposed = triplaze_rooms_df.transpose()
    print(tabulate(triplaze_rooms_transposed, headers="keys", tablefmt="pretty"))

    expedia_rooms = {}
    for code, room_obj in expedia_hotel["roomsMap"].items():
        expedia_rooms[room_obj["roomId"]] = room_obj["name"]

    expedia_rooms_df = pd.DataFrame([expedia_rooms])
    expedia_rooms_transposed = expedia_rooms_df.transpose()
    print("EXPEDIA HOTEL")
    print(tabulate(expedia_rooms_transposed, headers="keys", tablefmt="pretty"))


def get_triplaze_hotel_inventory(start_date, end_date, hotel_code):
    url = f"https://pyt-extapi-live.tiniva.com/api/inventoryPrices/viewInventory?productId={hotel_code}&startDate={start_date}&endDate={end_date}"
    payload = {}
    headers = {
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
        "Cache-Control": "no-cache",
        "Connection": "keep-alive",
        "Origin": "https://pyt-ext-live.tiniva.com",
        "Pragma": "no-cache",
        "Referer": "https://pyt-ext-live.tiniva.com/",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-site",
        "Sec-GPC": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
        "jwt": TRIPLAZE_JWT,
        "sec-ch-ua": '"Not/A)Brand";v="8", "Chromium";v="126", "Brave";v="126"',
        "sec-ch-ua-mobile": "?0",
        "sec-ch-ua-platform": '"macOS"',
    }
    response = requests.request("GET", url, headers=headers, data=payload)
    data = json.loads(response.text)["data"]
    sorted_data = []
    for room_data in data:
        key_list = room_data["dateWiseInventory"].keys()
        dates = [datetime.strptime(key, "%Y-%m-%d") for key in key_list]
        dates.sort()
        dates_sorted = [date.strftime("%Y-%m-%d") for date in dates]
        room_data_sorted = OrderedDict()
        room_data_sorted["dateWiseInventory"] = OrderedDict()
        for date in dates_sorted:
            room_data_sorted["dateWiseInventory"][date] = room_data[
                "dateWiseInventory"
            ][date]
            sorted_data.append(room_data_sorted)
    return sorted_data


def get_inventory(hotel_codes):
    start_date = datetime.now().strftime("%Y-%m-%d")
    end_date = (datetime.now() + timedelta(days=30)).strftime("%Y-%m-%d")
    counter = 0
    inventory_by_room = {}
    for hotel in hotel_codes:
        print(f"Getting inventory for hotel: {hotel}")
        inventory_data = get_triplaze_hotel_inventory(start_date, end_date, hotel)
        if inventory_data is None:
            print(f"Inventory not found for hotel: {hotel}")
        else:
            for room in inventory_data:
                counter = 0
                dates = []
                for date, inventory in room["dateWiseInventory"].items():
                    if (inventory["available"] > 0) and not inventory["block"]:
                        counter += 1
                        dates.append(date)
                        if counter > 2:
                            inventory_by_room = [dates, inventory["available"]]
                            break
                    else:
                        counter = 0
                        dates = []
        print(f"Inventory by room: {inventory_by_room}")
    return inventory_by_room

### Run the below cell to execute the hotel addition to the environment

In [ ]:
hotel_codes = ["3905","3876"]

environment = 'rewards'

uri = get_uri_for_environment(environment)
veho_uri = get_uri_for_veho_environment(environment)

client = pymongo.MongoClient(uri)
veho_client = pymongo.MongoClient(veho_uri)
print(f'{len(hotel_codes)} hotels to be added to {environment} database')

### Execute

In [ ]:
add_hotels_to_db(hotel_codes, environment)

### Validate Hotels added in Triplaze Collection

In [ ]:
triplaze_hotel_collection = client.get_database("hotels").get_collection(
    "triplaze_hotel"
)

triplaze_hotels = triplaze_hotel_collection.find(
    {"hotelCode": {"$in": hotel_codes}},
    {
        "_id": 0,
        "hotelCode": 1,
        "address": 1,
        "cityName": 1,
        "name": 1,
        "latitude": 1,
        "longitude": 1,
        "blendingParams": 1,
        "active": 1,
    },
)
triplaze_hotels = [hotel for hotel in triplaze_hotels]
print(f'Added {len(triplaze_hotels)} hotels to {environment} database')
print("TRIPLAZE HOTELS")
print(tabulate(triplaze_hotels, headers="keys", tablefmt="pretty"))

get_portal_status(hotel_codes)

### Activate Inactive Hotels in Triplaze

In [ ]:
# inactive_hotels = [3922]
# for inactive_hotel in inactive_hotels:
#     print(activate_triplaze_hotel(str(inactive_hotel)))

### Validate hotels added in hotel collection

In [ ]:
hotels = client.get_database('hotels').get_collection('hotel').find(
    {
        'hotelCodeBySourceProvider.TRIPLAZE' : {'$in': hotel_codes}
    },{
        '_id': 0,
        'planningToolId' : 1,
        'name':1,
        'cityName': 1,
        'hotelCodeBySourceProvider': 1,
        'distance': 1,
        'status': 1,
        'pytExclusive': 1,
        'pytRecommended': 1,
    }
)

hotels = [hotel for hotel in hotels]
print(f'{len(hotels)} hotels found in {environment} database')
print(tabulate(hotels, headers='keys', tablefmt='pretty'))

### Generate Fuzzy Script

In [ ]:
for city in get_distinct_cities(hotel_codes, environment):
    print(f"City: {city}")
    generate_fuzzy_sheet(city)
    generate_db_script(city)
    print("\n\n")

### Update Exclusivity and Status

In [ ]:
result = client.get_database('hotels').get_collection('hotel').update_many(
    {
        'hotelCodeBySourceProvider.TRIPLAZE' : {'$in': hotel_codes},
    },
    {
        '$set': {
            'status': 'y',
            'pytExclusive': True,
        }
        
    }
)

print(tabulate([result.raw_result], headers='keys', tablefmt='pretty'))

### Update Blending Params

In [ ]:
get_rooms_map(hotel_codes)
print(f'Expedia Hotel Code: {expedia_hotel_code}')

In [ ]:
roomsMap = {
  "12337": "200577682"
}

In [ ]:
result = client.get_database('hotels').get_collection('triplaze_hotel').update_many(
    {
        'hotelCode': hotel_codes[0]
    },
    {
        '$set': {
            'blendingParams.sourceProvider' : 'EXPEDIA',
            'blendingParams.hotelCode' : expedia_hotel_code,
            'blendingParams.blendLevel' : 'FULL_BLEND',
            'blendingParams.roomsMap': roomsMap
        }
    }
)

print(tabulate([result.raw_result], headers='keys', tablefmt='pretty'))

### Use Banana to clear cache 

In [ ]:
#Clear Search Pack Containers
def clear_searchpacks(city, environment):
    print(f"Clearing searchpacks for city {city} in {environment} environment")
    url = f"http://localhost:8000/cache/clear?city_name={city}&environment={environment}"
    response = requests.delete(url)
    print(f"API Call response: {response.text}, status code: {response.status_code}")

cities = get_distinct_cities(hotel_codes, environment)
for city in cities:
    clear_searchpacks(city, environment)

### City refresh for new hotel additions

In [ ]:
provider = 'TRIPLAZE'
cities_docs = veho_client.get_database('oceanjar').get_collection('city').find({'name' : {'$in' : cities}})

city_payloads = []
for city in cities_docs:
    city_payload = {
        'cityName' : city['name'],
        'cityLatitude' : city['latitude'],
        'cityLongitude' : city['longitude'],
        'provider' : provider,
        'reComputeDistance' : False,
        'providerCityId' : city['planningid'],
        'minimumDistanceFromHotel' : city['minimumDistanceFromHotel']
    }
    city_payloads.append(city_payload)

for payload in city_payloads:
    print(f'Refreshing city {payload["cityName"]}')
    refresh_hotels_for_city(payload, environment)


### Get Triplaze Inventory

In [ ]:
result = get_inventory(hotel_codes)
print(result)

dates = result[0]
checkin_date = dates[0]
checkout_date = dates[-1]
cities = get_distinct_cities(hotel_codes, environment)

### API Results from Hotel Search API

In [ ]:
if(environment == 'production'):
    url = "https://hotels.pickyourtrail.com/hotel/search"
else:
    url = f"http://{environment}.wwmibprivate.com:9090/hotel/search"

def convert_date(date_str):
    # Parse the date string to a datetime object
    date_obj = datetime.strptime(date_str, "%Y-%m-%d")
    # Convert the datetime object to the desired format
    formatted_date = date_obj.strftime("%d/%b/%Y")
    return formatted_date

payload_data = {
    "checkInDate": convert_date(checkin_date),
    "checkOutDate": convert_date(checkout_date),
    "passengerConfiguration": [{"adultCount": 2, "childAges": []}],
    "hotelPlanningIds": [],
    "cityName": cities[0],
    "minDistanceFromHotel": 30,
    "sourceProvider": "TRIPLAZE",
    "finalRoomCost": 0,
    "nationality": "IN",
    "topHotelsOnly": False,
    "globalInternational": False,
    "corporateBooking": False,
    "affiliateCode": "pyt",
    "mobileRequest": False,
    "productSuite": "b2c",
    "countryCode": "THA",
    "derbyInventoryNeeded": False,
    "cityFilterRequired": False,
    "limit": 0,
}
payload = json.dumps(payload_data)
headers = {
  'Content-Type': 'application/json',
}

response = requests.request("POST", url, headers=headers, data=payload)
data = json.loads(response.text)
if data is not None:
    for hotel in data['results']:
        hotel_data = {
            'hotelCode': hotel['hotelCode'],
            'name': hotel['name'],
            'planning id': hotel['planningId'],
            'city' : hotel['city'],
        }
        print(tabulate([hotel_data], headers='keys', tablefmt='pretty'))
